In [156]:
%reset -sf

In [18]:
# LOAD THIS BEFORE START, run rmdl

#!/usr/bin/env python3
import sys
import math, random
import functools, itertools, collections, heapq, bisect
from collections import Counter, defaultdict, deque

# import numpy as np
# import scipy
import matplotlib.pyplot as plt

abc = "abcdefghijklmnopqrstuvwxyz"
abc_map = {c:i for i,c in enumerate(abc)}
M9 = 10**9 + 7  # 998244353
yes, no = "YES", "NO"
d4 = [(1,0),(0,1),(-1,0),(0,-1)]
d8 = [
    (-1,-1),(-1,0),(-1,1),
    (0,-1),(0,0),(0,1),
    (1,-1),(1,0),(1,1)]
d6 = [(2,0),(1,1),(-1,1),(-2,0),(-1,-1),(1,-1)]  # hexagonal layout
MAXINT = sys.maxsize

In [103]:
with open("/Users/htong/Downloads/input.txt") as f:
    srr = f.read().strip().split("\n")

In [104]:
# srr = """
# Valve AA has flow rate=0; tunnels lead to valves DD, II, BB
# Valve BB has flow rate=13; tunnels lead to valves CC, AA
# Valve CC has flow rate=2; tunnels lead to valves DD, BB
# Valve DD has flow rate=20; tunnels lead to valves CC, AA, EE
# Valve EE has flow rate=3; tunnels lead to valves FF, DD
# Valve FF has flow rate=0; tunnels lead to valves EE, GG
# Valve GG has flow rate=0; tunnels lead to valves FF, HH
# Valve HH has flow rate=22; tunnel leads to valve GG
# Valve II has flow rate=0; tunnels lead to valves AA, JJ
# Valve JJ has flow rate=21; tunnel leads to valve II
# """.strip().split("\n")

In [105]:
# srr = """
# """.strip()

In [106]:
valve_to_pressure = {}
valve_to_nex = {}

for row in srr:
    row = row.replace("=", "|")
    row = row.replace(";", "|")
    row = row.replace(",", "")
    row = row.replace("valves", "|")
    row = row.replace("valve", "|")
    a,b,c,d = row.split("|")
    a = a.split()[1]
    b = int(b)
    nexs = d.split()
    valve_to_pressure[a] = b
    valve_to_nex[a] = nexs

In [107]:
# valve_to_pressure

In [108]:
# states = defaultdict(int)
# states["AA", tuple(), 0] = 0 # location, opened, currate -> value
# # prevmax = defaultdict(int)

# for _ in range(30):
#     print(_, len(states))
#     new_states = defaultdict(int)
#     for (cur, opened, currate), value in states.items():
#         if cur not in opened and valve_to_pressure[cur] > 0:
#             nex_opened = tuple(sorted(list(opened) + [cur]))
#             nex_currate = currate + valve_to_pressure[cur]
#             nex_state = (cur, nex_opened, nex_currate)
#             nex_value = value + currate
#             if nex_state not in states or nex_value > states[nex_state]:
#                 new_states[nex_state] = max(new_states[nex_state], nex_value)
#         for nex in valve_to_nex[cur]:
#             nex_state = (nex, opened, currate)
#             nex_value = value + currate
#             if nex_state not in states or nex_value > states[nex_state]:
#                 new_states[nex_state] = max(new_states[nex_state], nex_value)
#     states = new_states

In [110]:
states = defaultdict(int)
states["AA", "AA", tuple(), 0] = 0 # location1, location2, opened, currate -> value
# prevmax = defaultdict(int)

for _ in range(26):
    maxval = max(states.values())
    print(_, len(states), maxval)
    new_states = defaultdict(int)
    for (cur1, cur2, opened, currate), value in states.items():
        if value < maxval - 100:
            continue
        if cur1 not in opened and valve_to_pressure[cur1] > 0 and cur2 not in opened and valve_to_pressure[cur2] > 0 and cur1 != cur2:
            nex_opened = tuple(sorted(list(opened) + [cur1, cur2]))
            nex_currate = currate + valve_to_pressure[cur1] + valve_to_pressure[cur2]
            nex_state = (cur1, cur2, nex_opened, nex_currate)
            nex_value = value + currate
            if nex_state not in states or nex_value > states[nex_state]:
                new_states[nex_state] = max(new_states[nex_state], nex_value)

        if cur1 not in opened and valve_to_pressure[cur1] > 0:
            for nex2 in valve_to_nex[cur2]:
                nex_opened = tuple(sorted(list(opened) + [cur1]))
                nex_currate = currate + valve_to_pressure[cur1]
                nex_state = (cur1, nex2, nex_opened, nex_currate)
                nex_value = value + currate
                if nex_state not in states or nex_value > states[nex_state]:
                    new_states[nex_state] = max(new_states[nex_state], nex_value)

        if cur2 not in opened and valve_to_pressure[cur2] > 0:
            for nex1 in valve_to_nex[cur1]:
                nex_opened = tuple(sorted(list(opened) + [cur2]))
                nex_currate = currate + valve_to_pressure[cur2]
                nex_state = (nex1, cur2, nex_opened, nex_currate)
                nex_value = value + currate
                if nex_state not in states or nex_value > states[nex_state]:
                    new_states[nex_state] = max(new_states[nex_state], nex_value)
                    
        for nex1 in valve_to_nex[cur1]:
            for nex2 in valve_to_nex[cur2]:
                nex_state = (nex1, nex2, opened, currate)
                nex_value = value + currate
                if nex_state not in states or nex_value > states[nex_state]:
                    new_states[nex_state] = max(new_states[nex_state], nex_value)
    states.update(new_states)

0 1 0
1 26 0
2 61 0
3 474 0
4 1752 23
5 4040 46
6 11155 69
7 30349 131
8 55936 193
9 71410 255
10 80538 317
11 93891 379
12 115364 441
13 144353 516
14 181714 591
15 233076 696
16 268495 805
17 301722 930
18 312663 1055
19 329452 1180
20 346810 1311
21 367825 1446
22 394802 1587
23 419844 1728
24 457337 1883
25 497268 2041


In [111]:
max(states.values())

2213

In [ ]:
res = 0

In [266]:
def test(arr):
    res = 0
    return res